<a href="https://colab.research.google.com/github/shivam361v/DataScience/blob/main/KNN_image_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import libraries
import numpy as np
import cv2
from sklearn.neighbors import NearestNeighbors

In [7]:

# Load the low-resolution image and the training set
img = cv2.imread('low_res_image.jpg', cv2.IMREAD_GRAYSCALE)
F = np.load('feature_vectors.npy')
Y = np.load('pixel_values.npy')


FileNotFoundError: [Errno 2] No such file or directory: 'feature_vectors.npy'

In [ ]:

# Define the functions
def extract_feature_vector(img, i, j, n):
    # Extract a feature vector of size n*n from the local window around pixel (i,j)
    h, w = img.shape
    f = np.zeros((n*n,))
    for k in range(n):
        for l in range(n):
            x = i + k - n//2
            y = j + l - n//2
            if 0 <= x < h and 0 <= y < w:
                f[k*n + l] = img[x,y]
    return f

def find_k_nearest_neighbors(f, F, Y, t):
    # Find the k nearest neighbors of f in F using the Euclidean distance and the similarity threshold
    nbrs = NearestNeighbors(n_neighbors=len(F), metric='euclidean').fit(F)
    distances, indices = nbrs.kneighbors([f])
    k = np.argmax(distances > t)
    if k == 0:
        k = len(F)
    return F[indices[0][:k]], Y[indices[0][:k]], distances[0][:k]

def compute_weighted_MMSE_estimate(f, F, Y, d, sigma):
    # Compute the weighted MMSE estimate of the pixel value using the neighbors and their distances
    w = 1 / (sigma**2 + d**2)
    w = w / np.sum(w)
    x = np.dot(w, Y)
    return x

def apply_MRF_model(img, i, j, x, lambda, beta):
    # Apply the MRF model using the neighboring pixels and the potential function
    h, w = img.shape
    N = [(i-1,j), (i+1,j), (i,j-1), (i,j+1)] # 4-neighborhood
    E = lambda * np.sum([np.exp(-(x - img[x,y])**2 / (2 * beta**2)) for x,y in N if 0 <= x < h and 0 <= y < w])
    x = x + E
    return x



In [ ]:
# Loop over each pixel and interpolate
img_h = np.zeros((img.shape[0]*2, img.shape[1]*2)) # high-resolution image
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        # Extract the feature vector
        f = extract_feature_vector(img, i, j, n)
        # Find the k nearest neighbors
        F, Y, d = find_k_nearest_neighbors(f, F, Y, t)
        # Compute the weighted MMSE estimate
        x = compute_weighted_MMSE_estimate(f, F, Y, d, sigma)
        # Apply the MRF model
        x = apply_MRF_model(img, i, j, x, lambda, beta)
        # Assign the interpolated pixel value
        img_h[i*2, j*2] = x

# Save the high-resolution image
cv2.imwrite('high_res_image.jpg', img_h)
